In [4]:
import theano
import theano.tensor as T
import lasagne
import numpy as np

from load_mnist import *

In [ ]:
def batchify(X):
    

In [27]:
def load_data():
    X_train, y_train = load_mnist('training')
    X_test, y_test = load_mnist('testing')
    X_train = np.reshape(X_train, (len(X_train), 1, 28, 28))
    X_test = np.reshape(X_test, (len(X_test), 1, 28, 28))
    return X_train, y_train, X_test, y_test

In [31]:
X_train, y_train, X_test, y_test = load_data()

In [2]:
X = T.tensor4('X')
y = T.ivector('y')

In [ ]:
input_layer = lasagne.layers.InputLayer(shape=(None, 1, 28, 28), input_var=X),
#channel = 1 as images are black-white
input_layer_drop = lasagne.layers.DropoutLayer(input_layer, p=0.2)
    
hidden_layer_1 = lasagne.layers.DenseLayer(input_layer_drop, 
                                           num_units=800, 
                                           nonlinearity=lasagne.nonlinearities.rectify, 
                                           W=lasagne.init.GlorotUniform()
                                          )
hidden_layer_1_drop = lasagne.layers.DropoutLayer(hidden_layer_1, p=0.5)
    
hidden_layer_2 = lasagne.layers.DenseLayer(hidden_layer_1_drop, 
                                           num_units=800, 
                                           nonlinearity=lasagne.nonlinearities.rectify, 
                                           W=lasagne.init.GlorotUniform()
                                          )
hidden_layer_2_drop = lasagne.layers.DropoutLayer(hidden_layer_2, p=0.5)
    
output_layer = lasagne.layers.DenseLayer(hidden_layer_2_drop, 
                                         num_units=10, 
                                         nonlinearity=lasagne.nonlinearities.softmax
                                        )
    
prediction = lasagne.layers.get_output(output_layer)
loss = lasagne.objectives.categorical_crossentropy(prediction, y)
loss = loss.mean()
params = lasagne.layers.get_all_params(output_layer, trainable=True)
updates = lasagne.updates.sgd(loss, params, learning_rate=0.01)
#Used Stochastic Gradient Descent here without momentum
    
validation_prediction = lasagne.layers.get_output(output_layer, deterministic=True)
#We disable dropout for validation
validation_loss = lasagne.objectives.categorical_crossentropy(validation_prediction, y)
validation_loss = validation_loss.mean()
    
validation_accuracy = T.mean(T.eq(T.argmax(validation_prediction, axis=1), y), 
                      dtype=theano.config.floatX)

train_mlp = theano.function([X, y], loss, updates=updates)
validation_mlp = theano.function([X, y], [validation_loss, validation_accuracy])
#No updates performed in validation_mlp